In [ ]:
#!pip install paho-mqtt

In [2]:
# To talk with the MQTT server from the Discord Server
import paho.mqtt.client as mqtt

In [3]:
# Import to query wikipedia
#from langchain.utilities import WikipediaAPIWrapper
import wikipedia
from langchain.utilities import WikipediaAPIWrapper

In [4]:
from langchain import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [5]:
import requests

In [6]:
# Constants definitions
MQTT_BROKER = "192.168.72.148"  # MQTT IP Address
MQTT_PORT = 1883  # MQTT Port
MQTT_TOPIC = "Marvin"  # MQTT Topic 
MQTT_TOPIC_RECEIVE = "MarvinGenerated"  # MQTT Topic 
MQTT_USERNAME = "marvin"  # username
MQTT_PASSWORD = "secret"  # password

In [7]:
# Wikipedia setup in French
wikipedia.set_lang('fr')

In [8]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [9]:
# Wikipedia function to get a summary for a subject
def get_wikipedia_answer(Question):
    wiki_summ=wikipedia.search(Question)
    print(wiki_summ)
    return wiki_summ

In [10]:
def create_response(Question):
    llm = LlamaCpp(model_path="../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin",temperature=0.2,n_ctx=2048,max_tokens=1024)#,n_ctx=n_ctx,max_tokens=max_tokens
    template = """
    Tu es un assistant expert dans les domaines demandés. Tu réponds avec précision et fiabilité
    Réponds en français uniquement.
    {Question}
    """
    prompt = PromptTemplate(template=template, input_variables=["Question"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    reponse=llm_chain.predict(Question=Question)
    return(" **D'après mon modèle**\n"+reponse)


In [16]:
def create_wiki_response(Question):
    reponse = ''
    wikipedia = WikipediaAPIWrapper()
    wiki_summ=get_wikipedia_answer(Question)
    llm = LlamaCpp(model_path="../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin",temperature=0.2,n_ctx=1024,max_tokens=512)#,n_ctx=n_ctx,max_tokens=max_tokens)
    if wiki_summ != []:
        print(wiki_summ[0])
        template = """
            {content}
            {Question} 
        """
        prompt = PromptTemplate(template=template, input_variables=["Question","content"])
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        sujet=wiki_summ[0].replace(" ","_")
        page_summ = requests.get(f'https://fr.wikipedia.org/api/rest_v1/page/summary/{sujet}').json()["extract"]
        reponse=llm_chain.predict(wiki_summ=wiki_summ,Question=Question,content=page_summ)
        #reponse = wiki_summ[0]
        #print(dir(wikipedia.wiki_client.page(wiki_summ[0])))
        reponse = "\n \n **-------------------**\n" + reponse + "\n*Liens Wikipedia*\n"
        for page in wiki_summ:
            print(page)
            try:
                wikiurl=wikipedia.wiki_client.page(page).url
            except:
                wikiurl=''
            reponse= reponse + '\n' + wikiurl
    return(reponse)

In [12]:
#
# callback function called each time a message is received on the topic
#
def on_message(client, userdata, message):
    print("Received message on topic {}: {}".format(message.topic, message.payload.decode()))
    publish_response(message.payload.decode())

In [13]:
#
# We publish a message on another topic so the nodered
# can retrieve our answer
#
def publish_response(message):
    #
    # Placeholder to generate a response
    #
    #response = get_wikipedia_answer(message)
    response = create_response(message) + create_wiki_response(message)
    #response = create_wiki_response(message)
    client.publish(MQTT_TOPIC_RECEIVE, response)
    print(response)

In [14]:
#create_response('')

In [ ]:
#
# Main loop
#
# Client MQTT initialisation
client = mqtt.Client()
client.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)  
client.on_message = on_message
client.connect(MQTT_BROKER, MQTT_PORT)
client.subscribe(MQTT_TOPIC)

# Loop to receive messages
client.loop_forever()

Received message on topic Marvin: C'est quoi le théorème central limite ?


llama.cpp: loading model from ../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  59.11 KB
llama_model_load_internal: mem required  = 5809.32 MB (+ 2052.00 MB per state)
AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 
llama_init_from_file: kv self size  = 2048.00 MB

llama_print_timings:        lo

['Théorème central limite', 'Théorème du minimax de von Neumann', 'Intervalle de confiance', 'Théorème des deux carrés de Fermat', "Équipartition de l'énergie", 'Trou noir', 'Pythagore', 'Problème de Monty Hall', 'Emmy Noether', 'Pile ou face']
Théorème central limite


llama_init_from_file: kv self size  = 1024.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 

llama_print_timings:        load time =   556.40 ms
llama_print_timings:      sample time =    69.19 ms /   116 runs   (    0.60 ms per run)
llama_print_timings: prompt eval time =  5655.49 ms /    98 tokens (   57.71 ms per token)
llama_print_timings:        eval time = 18189.02 ms /   115 runs   (  158.17 ms per run)
llama_print_timings:       total time = 23928.19 ms


Théorème central limite
Théorème du minimax de von Neumann
Intervalle de confiance
Théorème des deux carrés de Fermat
Équipartition de l'énergie
Trou noir
Pythagore
Problème de Monty Hall
Emmy Noether
Pile ou face
 **D'après mon modèle**

    Le théorème central limite affirme que si une série de données suit une loi normale, alors la moyenne et l'écart type des valeurs de cette série sont approximativement proportionnels à la variance.
 
 **-------------------**

        Le théorème central limite est un résultat mathématique qui affirme que la somme de variables aléatoires indépendantes et identiquement distribuées tend vers une variable aléatoire gaussienne. C'est-à-dire, si vous ajoutez des quantités aléatoires simplement en fonction du temps (comme le nombre de pièces d'un certain type que vous avez acquises dans un certain mois), la somme des quantités tend vers une distribution normale.
*Liens Wikipedia*

https://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_central_limite
https://fr

llama.cpp: loading model from ../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  59.11 KB
llama_model_load_internal: mem required  = 5809.32 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  = 2048.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 

llama_print_timings:        lo

['Raspberry Pi', 'RISC-V', 'Universal Plug and Play', 'Kiwix', 'Liste des adoptants de GNU/Linux']
Raspberry Pi


llama.cpp: loading model from ../../twitch/vigogne/llama.cpp/models/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 1024
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  59.11 KB
llama_model_load_internal: mem required  = 5809.32 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  = 1024.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 

llama_print_timings:        lo

Raspberry Pi
RISC-V
Universal Plug and Play
Kiwix
Liste des adoptants de GNU/Linux
 **D'après mon modèle**

    # Code Python pour se connecter à Internet par Wi-Fi 
    import socket  
    s = socket.socket()  
    s.connect(('wifi', 1), 5)  
    print("Se connecté")
 
 **-------------------**

    """
    
    def __init__(self, mac_address=None):
        self._mac_address = mac_address
        if not self._mac_address:
            raise ValueError('No MAC address provided')
        
    def connect(self, ssid, password):
        """Connect to the given WiFi network"""
        
        try:
            wifi.sta.disconnect()
            
        except AttributeError:
            pass
        
        try:
            wifi.sta.config(ssid=ssid, password=password)
            
        except ValueError:
            raise ValueError('WiFi SSID or password invalid')
        
        self._mac_address = wifi.sta.get_station()
        
    def disconnect(self):
        """Disconnect from t